### Cross Chain Swap Notebook

#### setup the providers and addresses

In [74]:
import pandas as pd
from web3 import Web3
from web3.middleware import geth_poa_middleware
import json
import base64
import requests

provider = "https://red-icy-yard.matic-testnet.quiknode.pro/c58ed7061d2badc4cac809cf43acb81a9a17af69/"
erc20_abi = json.load(open('./ERC20.json'))
dCast_abi = json.load(open('../artifacts/contracts/CCS.sol/CrossChainSwap.json'))['abi']


web3 = Web3(Web3.HTTPProvider(provider))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [72]:
WMATIC_address =  "0x9c3C9283D3e44854697Cd22D3Faa240Cfb032889"
WETH_goerli_address = "0xB4FBF271143F4FBf7B91A5ded31805e42b2208d6"
USDC_mumbai_address = "0xA6FA4fB5f76172d178d61B04b0ecd319C5d1C0aa"


owner = "0x0D80C4B7713Cb4a4A2A38c0F680f7e71d6524B88"
owner_pk = "9178e0d78d0f8ea9b267274f62574d1347e97430c062e5b7d6ed9f2483e9dee5"

contract_address = "0xAC628B2C8CdBd7F358c9E46F9E987fB22DEf64dB"

In [69]:
wmatic = web3.eth.contract(address=WMATIC_address, abi = erc20_abi)
usdc = web3.eth.contract(address=USDC_mumbai_address, abi = erc20_abi)
contract = web3.eth.contract(address=contract_address, abi = dCast_abi)

#### approving contract to spend native tokens

In [70]:
txn = wmatic.functions.approve(contract_address, 1000000).buildTransaction(
    {
        'from': owner,
        'nonce': web3.eth.get_transaction_count(owner),
        'gasPrice': 20000000000
    }
)

tx_create = web3.eth.account.sign_transaction(txn, owner_pk)

tx_hash = web3.eth.send_raw_transaction(tx_create.rawTransaction)
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

print(f'Tx successful with hash: { tx_receipt.transactionHash.hex() }')

Tx successful with hash: 0xa89317f87b8e3c14e396e862bbc13a8c49a01d50b671cbac629b5c3041d45cb5


In [71]:
txn = usdc.functions.approve(contract_address, 1000000).buildTransaction(
    {
        'from': owner,
        'nonce': web3.eth.get_transaction_count(owner),
        'gasPrice': 20000000000
    }
)

tx_create = web3.eth.account.sign_transaction(txn, owner_pk)

tx_hash = web3.eth.send_raw_transaction(tx_create.rawTransaction)
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

print(f'Tx successful with hash: { tx_receipt.transactionHash.hex() }')

Tx successful with hash: 0x5d2eaf0e7a5cfd65736acd7ea258c50b09de17a5b2d031a0efcb181a7cedae84


### mint_for_deposit
this function deposits USDC from your wallet and mints exact same amount of $CAST tokens 

In [64]:
txn = contract.functions.mint_for_deposit(10000).buildTransaction(
    {
        'from': owner,
        'nonce': web3.eth.get_transaction_count(owner),
        'gasPrice': 20000000000
    }
)

tx_create = web3.eth.account.sign_transaction(txn, owner_pk)

tx_hash = web3.eth.send_raw_transaction(tx_create.rawTransaction)
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

print(f'Tx successful with hash: { tx_receipt.transactionHash.hex() }')

Tx successful with hash: 0x5fe60a86e9cc2ce3dda5a6efafa5b256923697c3a60b529d83e71fcd8f1d9697


#### burn_for_withdrawal
this function burns $CAST tokens from your wallet and returns USDC (if available!)

In [ ]:
txn = contract.functions.burn_for_withdrawal(1000000).buildTransaction(
    {
        'from': owner,
        'nonce': web3.eth.get_transaction_count(owner),
        'gasPrice': 20000000000
    }
)

tx_create = web3.eth.account.sign_transaction(txn, owner_pk)

tx_hash = web3.eth.send_raw_transaction(tx_create.rawTransaction)
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

print(f'Tx successful with hash: { tx_receipt.transactionHash.hex() }')

#### CCSwapStart
this function creates new swap based on source and destination tokens and returns Wormhole VAA

In [61]:
txn = contract.functions.CCSwapStart(WMATIC_address, WETH_goerli_address, 500, 10000).buildTransaction(
    {
        'from': owner,
        'nonce': web3.eth.get_transaction_count(owner),
        'gasPrice': 20000000000
    }
)

tx_create = web3.eth.account.sign_transaction(txn, owner_pk)

tx_hash = web3.eth.send_raw_transaction(tx_create.rawTransaction)
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

print(f'Tx successful with hash: { tx_receipt.transactionHash.hex() }')

Tx successful with hash: 0x15077f2bf757169202661bbd1ad4bd2607c022c2a13a0e1724dfec0b8ee8b7a6


### get signedVAA
fetches signed VAA from Wormhole

In [ ]:
wormhole_bridge = "0x0CBE91CF822c73C2315FB05100C2F714765d5c20"

recipt = web3.eth.get_transaction_receipt(tx_hash)
for log in web3.eth.get_transaction_receipt("0x7a629f5af2d391ae553716c4fcec94c1578c57babf55300c4e7d28b78aefe588")["logs"]:
    if(log['address']==wormhole_bridge):
        sequenceId = int(log['data'][2:66])
        response = requests.get("https://wormhole-v2-testnet-api.certus.one/v1/signed_vaa/5/000000000000000000000000"+contract_address[2:]+"/"+str(sequenceId)).json()['vaaBytes']
        print(web3.toHex(bytearray(base64.b64decode(response))))

#### CCSwapEnd
this function is called on the destination chain with signed VAA to complete the cross chain swap

In [280]:
provider = "https://red-icy-yard.matic-testnet.quiknode.pro/c58ed7061d2badc4cac809cf43acb81a9a17af69/"

web3 = Web3(Web3.HTTPProvider(provider))
contract = web3.eth.contract(address=contract_address, abi = dCast_abi)

txn = contract.functions.CCSwapEnd("0x01000000000100ce873470d8672d9883269611fcbfc8a839b4047dbd4ce705aec3ff4035490af15df1c9b100a73da8850acafda9b0f3525ed81384250bede8cb51b4b3ed3dd48b0163e549190000000000050000000000000000000000008641c2d322ac60a659ecbadcf8cb817a82bf8cd70000000000000000c80000000000000000000000000d80c4b7713cb4a4a2a38c0f680f7e71d6524b8800000000000000000000000000000000000000000000000000000000000000c70000000000000000000000009c3c9283d3e44854697cd22d3faa240cfb0328890000000000000000000000000000000000000000000000000000000000030d40", 500).buildTransaction(
    {
        'from': owner,
        'nonce': web3.eth.get_transaction_count(owner),
        'gasPrice': 20000000000
    }
)

tx_create = web3.eth.account.sign_transaction(txn, owner_pk)

tx_hash = web3.eth.send_raw_transaction(tx_create.rawTransaction)
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

print(f'Tx successful with hash: { tx_receipt.transactionHash.hex() }')

Tx successful with hash: 0x60959aa5452e26bbbde019589464346564f9f61e77e4b443f872de5f83be96b2
